In [3]:
import os
import pickle
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.spatial import Delaunay
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import networkx as nx
from itertools import combinations
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [4]:

def load_data(filepath: str) -> dict:
    """
    Load data from pickle file. The data is expected to be a dictionary:
    {node_identifier: pd.DataFrame with 'traffic_flow', 'Xkoordinat', 'Ykoordinat'}
    """
    with open(filepath, "rb") as f:
        data = pickle.load(f)
    return data


def split_dataframes(df_dict: dict, train_ratio=0.7, val_ratio=0.1):
    """
    Splits the dictionary of DataFrames into train, validation, and test sets.
    Splits are done by time sequence.

    Args:
        df_dict: {node_key: pd.DataFrame}
        train_ratio: fraction of data for training
        val_ratio: fraction of data for validation

    Returns:
        df_dict_train, df_dict_val, df_dict_test
    """
    df_dict_train, df_dict_val, df_dict_test = {}, {}, {}
    for key, df in df_dict.items():
        n = len(df)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))
        df_train = df.iloc[:train_end].copy()
        df_val = df.iloc[train_end:val_end].copy()
        df_test = df.iloc[val_end:].copy()

        df_dict_train[key] = df_train
        df_dict_val[key] = df_val
        df_dict_test[key] = df_test

    return df_dict_train, df_dict_val, df_dict_test


def create_sliding_window_data(df_dict: dict, lookback: int, pred_horizon: int):
    """
    Create sliding windows for each node.
    Returns {node: X_node}, {node: y_node}
    X_node shape: (#samples, lookback), y_node shape: (#samples, pred_horizon)
    """
    X_dict, y_dict = {}, {}
    for key, df in df_dict.items():
        node_series = df['traffic_flow'].values
        X_node, y_node = [], []
        for i in range(len(node_series) - lookback - pred_horizon + 1):
            X_node.append(node_series[i:i+lookback])
            y_node.append(node_series[i+lookback:i+lookback+pred_horizon])
        X_dict[key] = np.array(X_node)
        y_dict[key] = np.array(y_node)
    return X_dict, y_dict


def combine_node_data(X_dict: dict, y_dict: dict):
    """
    Combine data from all nodes into single arrays.
    """
    X_list = [X_dict[node] for node in X_dict]
    y_list = [y_dict[node] for node in y_dict]
    X_combined = np.concatenate(X_list, axis=0)
    y_combined = np.concatenate(y_list, axis=0)
    return X_combined, y_combined

In [5]:
def clean_graph(graph: nx.Graph) -> nx.Graph:
    """
    Clean the graph: remove self-loops, duplicate edges, keep largest component.
    """
    g = graph.copy()
    g.remove_edges_from(list(nx.selfloop_edges(g)))

    if len(g.nodes) > 0:
        largest_component = max(nx.connected_components(g), key=len)
        g = g.subgraph(largest_component).copy()

    return g

# =====================================================
#                   Graph Construction
# =====================================================

def create_distance_graph(df_dict, weighted=False, degree=4):
    """
    Create a graph where edges are based on spatial proximity.
    """
    graph = nx.Graph()
    coords = {}
    for key, df in df_dict.items():
        x, y = df['Xkoordinat'].iloc[0], df['Ykoordinat'].iloc[0]
        graph.add_node(key, x=x, y=y, traffic_flow=df['traffic_flow'].values)
        coords[key] = (x, y)

    # Compute all pairwise distances
    keys = list(df_dict.keys())
    dist_list = []
    for i, k1 in enumerate(keys):
        for k2 in keys[i+1:]:
            x1, y1 = coords[k1]
            x2, y2 = coords[k2]
            dist = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            dist_list.append((k1, k2, dist))

    # Sort edges by distance (ascending)
    dist_list.sort(key=lambda x: x[2])

    # Add edges while respecting degree constraints
    for k1, k2, dist in dist_list:
        if graph.degree[k1] < degree and graph.degree[k2] < degree:
            if weighted:
                # Normalize weights if desired
                graph.add_edge(k1, k2, weight=1/(1+dist))
            else:
                graph.add_edge(k1, k2)


    graph = clean_graph(graph)
    return graph


def create_delaunay_graph(df_dict):
    """
    Create a graph based on the Delaunay triangulation of spatial coordinates.

    Parameters:
        df_dict (dict): A dictionary where keys are node identifiers and values are
                        pandas DataFrames containing 'Xkoordinat' and 'Ykoordinat'.

    Returns:
        nx.Graph: A Delaunay graph where edges are formed from Delaunay triangulation.
    """
    # Initialize an empty graph
    graph = nx.Graph()

    # Extract node coordinates and their identifiers
    nodes = []
    coords = []
    for key, df in df_dict.items():
        x, y = df['Xkoordinat'].iloc[0], df['Ykoordinat'].iloc[0]
        graph.add_node(key, x=x, y=y, traffic_flow=df['traffic_flow'].values)
        nodes.append(key)
        coords.append((x, y))

    # Perform Delaunay triangulation
    tri = Delaunay(coords)

    # Add edges from the Delaunay triangulation
    for simplex in tri.simplices:
        for i in range(3):
            for j in range(i + 1, 3):
                node1 = nodes[simplex[i]]
                node2 = nodes[simplex[j]]
                graph.add_edge(node1, node2)

    # Clean the graph (remove self-loops, keep largest connected component, etc.)
    graph = clean_graph(graph)

    return graph

def create_correlation_graph(df_dict_train, df_dict_full, threshold=0.5, weighted=False, degree=6):
    """
    Create a correlation-based graph using training data for correlation calculation.
    """
    graph = nx.Graph()
    keys = list(df_dict_full.keys())
    for key, df in df_dict_full.items():
        graph.add_node(key, x=df['Xkoordinat'].iloc[0], y=df['Ykoordinat'].iloc[0],
                       traffic_flow=df['traffic_flow'].values)

    # Compute correlation
    corr_matrix = {}
    for i, k1 in enumerate(keys):
        for j, k2 in enumerate(keys[i+1:], i+1):
            corr = np.corrcoef(df_dict_train[k1]['traffic_flow'], df_dict_train[k2]['traffic_flow'])[0, 1]
            corr_matrix[(k1, k2)] = corr
            corr_matrix[(k2, k1)] = corr

    # Sort edges by correlation
    for k1 in keys:
        potential = [(k2, corr_matrix[(k1, k2)]) for k2 in keys if k1 != k2 and (k1, k2) in corr_matrix]
        potential.sort(key=lambda x: -x[1])  # descending order by correlation
        for k2, c in potential:
            if graph.degree[k1] < degree and graph.degree[k2] < degree and c > threshold:
                if weighted:
                    graph.add_edge(k1, k2, weight=c)
                else:
                    graph.add_edge(k1, k2)
    
    # find any nodes that are not connected to any other nodes, and connect
    graph = clean_graph(graph)
    return graph


def create_cosine_graph(df_dict, weighted=False, degree=4):
    """
    Create a graph based on cosine similarity between full node time series.
    """
    graph = nx.Graph()
    keys = list(df_dict.keys())
    features = [df_dict[k]['traffic_flow'].values for k in keys]
    features = np.array(features)  # shape: (#nodes, #timesteps)
    # Compute cosine similarity
    norm = np.linalg.norm(features, axis=1, keepdims=True)
    normed = features / (norm + 1e-8)
    sim_matrix = normed @ normed.T  # cosine similarity

    for i, k1 in enumerate(keys):
        graph.add_node(k1,
                       x=df_dict[k1]['Xkoordinat'].iloc[0],
                       y=df_dict[k1]['Ykoordinat'].iloc[0],
                       traffic_flow=df_dict[k1]['traffic_flow'].values)

    # Sort by similarity
    for i, k1 in enumerate(keys):
        potential = [(keys[j], sim_matrix[i, j]) for j in range(len(keys)) if i != j]
        potential.sort(key=lambda x: -x[1])
        for k2, sim_val in potential:
            if graph.degree[k1] < degree and graph.degree[k2] < degree:
                if weighted:
                    graph.add_edge(k1, k2, weight=sim_val)
                else:
                    graph.add_edge(k1, k2)
    graph = clean_graph(graph)
    return graph


def create_fully_connected_graph(df_dict, weighted=False):
    """
    Create a fully connected graph. Optionally assign random weights.
    """
    graph = nx.Graph()
    keys = list(df_dict.keys())
    for k in keys:
        df = df_dict[k]
        graph.add_node(k, x=df['Xkoordinat'].iloc[0], y=df['Ykoordinat'].iloc[0],
                       traffic_flow=df['traffic_flow'].values)

    for i, k1 in enumerate(keys):
        for k2 in keys[i+1:]:
            if weighted:
                graph.add_edge(k1, k2, weight=np.random.rand())
            else:
                graph.add_edge(k1, k2)
    graph = clean_graph(graph)
    return graph


def create_dynamic_weight_graph(df_dict, lookback, pred_horizon, adaptive_factor=1.0, max_degree=4):
    """
    Create a graph with dynamic weights based on correlation of sliding windows.
    """
    graph = nx.Graph()
    keys = list(df_dict.keys())
    for k in keys:
        df = df_dict[k]
        graph.add_node(k, x=df['Xkoordinat'].iloc[0], y=df['Ykoordinat'].iloc[0],
                       traffic_flow=df['traffic_flow'].values)

    # Precompute sliding windows
    sliding_windows = {
        k: np.lib.stride_tricks.sliding_window_view(df_dict[k]['traffic_flow'].values, lookback)[:-pred_horizon]
        for k in keys
    }

    # Compute avg correlation over time for each pair
    for i, k1 in enumerate(keys):
        for k2 in keys[i+1:]:
            w1 = sliding_windows[k1]
            w2 = sliding_windows[k2]
            # correlation per window
            corrs = []
            for t in range(min(len(w1), len(w2))):
                if np.std(w1[t]) > 0 and np.std(w2[t]) > 0:
                    c = np.corrcoef(w1[t], w2[t])[0, 1]
                    corrs.append(c)
            if len(corrs) > 0:
                avg_weight = np.mean(corrs)
                threshold = adaptive_factor * avg_weight
                if avg_weight > threshold:
                    graph.add_edge(k1, k2, weight=avg_weight)

    # Enforce max degree
    for node in list(graph.nodes()):
        edges = sorted(graph.edges(node, data=True), key=lambda x: x[2]['weight'], reverse=True)
        if len(edges) > max_degree:
            for edge in edges[max_degree:]:
                graph.remove_edge(edge[0], edge[1])

    graph = clean_graph(graph)
    return graph


def TOTAL_RANDOM_WEIGHTS(df_dict, degrees=32):
    # Initialize a graph
    num_nodes = len(df_dict)
    graph = nx.Graph()

    # Add nodes with attributes
    for i, (key, df) in enumerate(df_dict.items()):
        graph.add_node(i, x=df['Xkoordinat'].iloc[0], y=df['Ykoordinat'].iloc[0], traffic_flow=df['traffic_flow'].values)

    # Randomly add edges while respecting degree constraints
    nodes = list(graph.nodes)
    max_attempts = num_nodes * degrees * 10  # Safety limit to prevent infinite loops
    attempts = 0

    while any(graph.degree(n) < degrees for n in nodes):
        u, v = np.random.choice(nodes, size=2, replace=False)
        # Add an edge only if it doesn't exist and both nodes are under degree limit
        if not graph.has_edge(u, v) and graph.degree(u) < degrees and graph.degree(v) < degrees:
            graph.add_edge(u, v, weight=np.random.rand())
        
        attempts += 1
        if attempts >= max_attempts:
            print("Warning: Maximum attempts reached, stopping early.")
            break

    # Return the graph and edge weights
    return graph, nx.get_edge_attributes(graph, 'weight')

In [6]:

class MLPDecoder(nn.Module):
    def __init__(self, latent_channels):
        super().__init__()
        self.fc1 = nn.Linear(latent_channels * 2, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, z, edge_index):
        edge_embeddings = torch.cat([z[edge_index[0]], z[edge_index[1]]], dim=1)
        return torch.sigmoid(self.fc2(F.relu(self.fc1(edge_embeddings))))

    
class GraphAutoEncoder(nn.Module):
    """
    Graph Autoencoder with GCN-based encoder and inner product decoder.
    """
    def __init__(self, in_channels, hidden_channels, latent_channels):
        super(GraphAutoEncoder, self).__init__()
        # Encoder: GCN layers
        self.encoder1 = GCNConv(in_channels, hidden_channels)
        self.encoder2 = GCNConv(hidden_channels, latent_channels)
        # Decoder: Inner product decoder
        self.decoder = MLPDecoder(latent_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Encoder: Node feature transformations
        x = F.relu(self.encoder1(x, edge_index))
        z = self.encoder2(x, edge_index)
        # Decoder: Reconstruct probabilities for given edges
        recon_adj = self.decoder(z, edge_index)
        return z, recon_adj

# Updated loss function
def gae_loss(recon_adj, edge_index, data):
    """
    Reconstruction loss: Compare reconstructed probabilities with actual edges.
    """
    # Positive edges (existing edges in the graph)
    pos_mask = torch.ones_like(recon_adj)
    pos_loss = -torch.log(recon_adj + 1e-15).mean()

    # Negative edges
    num_nodes = data.num_nodes
    neg_edge_index = torch.randint(0, num_nodes, edge_index.size(), dtype=torch.long, device=edge_index.device)
    neg_loss = -torch.log(1 - recon_adj + 1e-15).mean()

    return pos_loss + neg_loss

# Training and Evaluation
def train_gae(model, data, epochs, lr, device):
    """
    Train the Graph Autoencoder model.
    """
    model.to(device)
    optimizer = Adam(model.parameters(), lr=lr)
    model.train()
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        z, recon_adj = model(data)
        loss = gae_loss(recon_adj, data.edge_index, data)
        loss.backward()
        optimizer.step()
        
        if epoch % 1 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

    return model

def evaluate_gae(model, data):
    """
    Evaluate the Graph Autoencoder: Compute reconstruction metrics.
    """
    model.eval()
    with torch.no_grad():
        z, recon_adj = model(data)
        # Convert reconstructed adjacency to binary predictions
        pred_adj = (recon_adj > 0.5).float()
        actual_adj = torch.zeros_like(pred_adj)
        actual_adj[data.edge_index[0], data.edge_index[1]] = 1.0
        # Compute metrics: Reconstruction accuracy
        correct = (pred_adj == actual_adj).sum().item()
        total = pred_adj.numel()
        accuracy = correct / total
        print(f"Reconstruction Accuracy: {accuracy:.4f}")
        return accuracy, z


DATA_PATH = 'data_prep/one_year.pkl'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOOKBACK = 50
PRED_HORIZON = 1
TRAIN_RATIO = 0.6
VAL_RATIO = 0.2


# Load and split data
final_dataframes = load_data(DATA_PATH)
df_dict_train, df_dict_val, df_dict_test = split_dataframes(final_dataframes, TRAIN_RATIO, VAL_RATIO)

# Create sliding window data
X_train_dict, y_train_dict = create_sliding_window_data(df_dict_train, LOOKBACK, PRED_HORIZON)
X_val_dict, y_val_dict = create_sliding_window_data(df_dict_val, LOOKBACK, PRED_HORIZON)
X_test_dict, y_test_dict = create_sliding_window_data(df_dict_test, LOOKBACK, PRED_HORIZON)

node_lengths = {node: X_train_dict[node].shape[0] for node in X_train_dict}

# Combine data
X_train, y_train = combine_node_data(X_train_dict, y_train_dict)
X_val, y_val = combine_node_data(X_val_dict, y_val_dict)
X_test, y_test = combine_node_data(X_test_dict, y_test_dict)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float, device=device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float, device=device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float, device=device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float, device=device)
# Example Usage

# Example Usage
DEGREE = 6
HIDDEN_CHANNELS = 32
in_channels = LOOKBACK  # Input features (sliding window size)
hidden_channels = HIDDEN_CHANNELS  # Hidden layer size
latent_channels = 16  # Latent space dimension
gae_model = GraphAutoEncoder(in_channels, hidden_channels, latent_channels)

# Prepare graph data
graph = create_distance_graph(final_dataframes, weighted=True, degree=DEGREE)
node_mapping = {node: i for i, node in enumerate(graph.nodes())}
edge_index = torch.tensor(
    [[node_mapping[u], node_mapping[v]] for u, v in graph.edges()],
    dtype=torch.long, device=device
).t().contiguous()
gae_data = Data(x=X_train_tensor, edge_index=edge_index, num_nodes=len(graph.nodes))

# Train GE
gae_model = train_gae(gae_model, gae_data, epochs=500, lr=1e-4, device=device)

# Evaluate GAE
accuracy, embeddings = evaluate_gae(gae_model, gae_data)

FileNotFoundError: [Errno 2] No such file or directory: 'data_prep/one_year.pkl'